In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

from astroLuSt.database_interface import alerceinterface
from astroLuSt.database_interface import eleanorinterface
from astroLuSt.database_interface import gaiainterface
from astroLuSt.database_interface import lightkurveinterface
from astroLuSt.database_interface import simbadinterface

import importlib
importlib.reload(alerceinterface)
importlib.reload(eleanorinterface)
importlib.reload(gaiainterface)
importlib.reload(lightkurveinterface)
importlib.reload(simbadinterface)

#style for plotting
plt.style.use('astroLuSt.styles.LuSt_style')

In [ ]:
targets = [
    "KIC 5006817", "RR Lyr", "TV Boo"
]

# SimbadDatabaseInterface

In [ ]:
SDI = simbadinterface.SimbadDatabaseInterface()
ids = SDI.get_ids(
    targets
)
tics = SDI.df_ids["TIC"]
print(tics)

# EleanorDatabaseInterface

In [54]:
importlib.reload(eleanorinterface)

EDI = eleanorinterface.EleanorDatabaseInterface(
    tics=tics[:], sectors="all",
    aperture_mode='large',
    )

EDI.data_from_eleanor_alltics(
    save="./data/",
    # save=True,
    redownload=False,
    plot_result=True, save_plot=False,
    sleep=0,
    n_jobs=1, n_chunks=1,
)


INFO: Extracting 1 chuncks with shapes [(3,)]
INFO: Working on chunk 1/1 with size 3

Extracting tic122447138 (chunk 1/1, 1/3 = 1/3 Total)

Extracting tic159717514 (chunk 1/1, 2/3 = 2/3 Total)

Extracting tic168709463 (chunk 1/1, 3/3 = 3/3 Total)

Extraction summary:
         TIC  sucess                             original error message
0  122447138   False  WARNING: ./data/tic122447138.csv already exist...
1  159717514   False  WARNING: ./data/tic159717514.csv already exist...
2  168709463   False  WARNING: ./data/tic168709463.csv already exist...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


,TIC,sucess,original error message
0,122447138,False,WARNING: ./data/tic122447138.csv already exist...
1,159717514,False,WARNING: ./data/tic159717514.csv already exist...
2,168709463,False,WARNING: ./data/tic168709463.csv already exist...


# GaiaDatabaseInterface

In [ ]:
GID = gaiainterface.GaiaDatabaseInterface()
GID.gaia_crendetials = "../credentials_gaia.txt"

filter = "(jobs['phase'] == 'ERROR')"
GID.remove_all_jobs(pd_filter=filter, login_before=False, logout_after=False)


# AlerceDatabaseInterface

In [ ]:

ADI = alerceinterface.AlerceDatabaseInterface()

df = pd.DataFrame(
    data=np.array([
        [10054,	12.39495833, 27.02213889,],
        [10088,	353.7751667, np.inf,],#41.10291667,],
        [10140,	16.294625,	 34.21841667,],
        [10147,	359.6756667, 41.48880556,],
    ]),
    columns=['id', 'ra', 'dec']
)

df_ztf = ADI.crossmerge_by_coordinates(
    df_left=df,
    ra_colname='ra', dec_colname='dec', radius=10,
    sleep=2E-3,
    n_jobs=1, verbose=2
)

print(ADI.df_error_msgs_crossmerge)

ADI.download_lightcurves(
    df_ztf['oid_ztf'],
    save=False,
    # save=alerce_extraction_dir,
    plot_result=True, save_plot=False, close_plots=False,
    sleep=2E-3,
    n_jobs=1, verbose=2
)
print(ADI.df_error_msgs_lcdownload)

# LightkurveInterface

In [ ]:

importlib.reload(lightkurveinterface)
LKI = lightkurveinterface.LightkurveInterface(
    tics=tics.astype(np.float64)
)

LKI.download_lightcurves_tess(
    #saving data
    save=False,
    sectors='all',
    quality_expression="(datum.quality == 0)",
    include_aperture=False, include_tpf=False,
    #plotting
    plot_result=True,
    aperture_detail=50, ylims=None,
    fontsize=16, figsize=(16,9),
    save_plot=False,
    sleep=0,
    n_jobs=1, n_chunks=1,
    verbose=2
)

print(LKI.df_extraction_summary)